In [126]:
import pandas as pd
from sqlalchemy import create_engine
import os
import shutil
import yaml
import re
import html
import urllib

In [2]:
with open('topic_links.yml', 'r') as f:
    topic_pagination = yaml.load(f.read(), Loader=yaml.FullLoader)

with open('post_links.yml', 'r') as f:
    post_pagination = yaml.load(f.read(), Loader=yaml.FullLoader)


In [3]:
def replace_simple_tags(txt, bb_uid):
    tags = {
        'u' : 'U',
        'b' : 'STRONG',
        'i' : 'EM',
        's' : 'STRIKE',
        'sup' : 'SUP',
        'sub' : 'SUB',
        'code' : 'PRE',
        '*' : 'LI'
    }
    for t_before, t_after in  tags.items():

        start_tag_before = f'[{t_before}:{bb_uid}]'
        start_tag_after = f'<{t_after}>'
        end_tag_before = f'[/{t_before}:{bb_uid}]'
        end_tag_before_m = f'[/{t_before}:m:{bb_uid}]'
        end_tag_before_u = f'[/{t_before}:u:{bb_uid}]'
        end_tag_before_o = f'[/{t_before}:o:{bb_uid}]'
        end_tag_after = f'</{t_after}>'
        
        txt = txt \
            .replace(start_tag_before, start_tag_after) \
            .replace(end_tag_before, end_tag_after) \
            .replace(end_tag_before_m, end_tag_after) \
            .replace(end_tag_before_u, end_tag_after) \
            .replace(end_tag_before_o, end_tag_after)
            
    return txt


In [4]:
def replace_linebreaks(txt, bb_uid):
    # first, remove excess lien breaks after selected end-tags
    rep1 =  r'\1' + f'[br:{bb_uid}]'
    txt = re.sub(f'(\[\/(list|quote|code|\*)(\:(m|o|u))?\:{bb_uid}\])\n', rep1, txt)
    # ... and after selected start tags
    txt = re.sub(f'(\[(list|quote|code|\*)(=[^\:]+)?\:{bb_uid}\])\n', rep1, txt)
    # then, change all other lien breaks to BRs
    txt = re.sub('(?<!\n)\n(?!\n)', '<BR />\n', txt)
    txt = re.sub('(?<!\n)\n\n+(?!\n)', '<BR />\n<BR />\n', txt)
    # then restore the linebreaks after selected tags for readability
    txt = re.sub(f'\[br\:{bb_uid}]', '\n', txt)
    return txt


In [81]:
def replace_quotes(txt, bb_uid):

    txt = re.sub(
        f'\[quote\=(?:\&quot\;)([^\&]+)(?:\&quot\;):{bb_uid}\]',
        r'<BLOCKQUOTE><P>\n»\1« pisze:<BR />\n',
        txt
    )

    txt = re.sub(
        f'\[quote\:{bb_uid}\]',
        r'<BLOCKQUOTE><P>\ncytat:<BR />\n',
        txt
    )

    txt = re.sub(
        f'\[\/quote(\:(o|u|m))?\:{bb_uid}\]',
        '\n</P></BLOCKQUOTE>',
        txt
    )

    return txt
    

In [18]:
def replace_lists(txt, bb_uid):
    txt = re.sub(
        f'\[list\=(.)\:{bb_uid}\]',
        r'<OL type="\1">',
        txt
    )
    txt = re.sub(
        f'\[list\:{bb_uid}\]',
        r'<UL>',
        txt
    )
    txt = re.sub(
        f'\[\/list\:o\:{bb_uid}\]',
        r'</OL>',
        txt
    )
    txt = re.sub(
        f'\[\/list\:u\:{bb_uid}\]',
        r'</UL>',
        txt
    )

    return txt

# test post for UL 34524
# test post for OL 
# mixed lists 23789, 31042  , 21737


In [65]:
def translate_url(url):

    new_url = url

    known_hosts = [
        'http://pwgay.org/forum/',
        'http://pwgay.7z9.net/forum/'
    ]
    m = 0
    for host_pattern in known_hosts:
        m += len(re.findall(host_pattern, url))

    if m >0 :

        param_pattern = '[a-zA-Z]+\=[0-9a-zA-Z]+'
        params = re.search(
            f'(?:viewtopic|vievforum).php\?((?:{param_pattern})(?:\&{param_pattern})*)',
            url
        )
        
        if params != None:

            param_arr = params.group(1).split('&')
            param_dict = { p.split('=')[0] : p.split('=')[1] for p in param_arr}

            post_id = param_dict.get('p')
            topic_id = param_dict.get('t')
            forum_id = param_dict.get('f')
            
            if post_id != None:

                link_params = post_pagination.get(int(post_id))
                thread_num = link_params['t']
                page_num = link_params['p']
                if page_num == 1:
                    new_url = f'/threads/{thread_num}/index.html#{post_id}'
                else:
                    new_url = f'/threads/{thread_num}/page_{page_num}.html#{post_id}'

            elif topic_id != None:

                new_url = f'/threads/{topic_id}/index.html'

            elif forum_id != None:

                new_url = f'/forums/{forum_id}/index.html'
    
    return new_url


In [38]:
def url_tag(url, make_tag='none'):
    url = html.unescape(url)
    url = translate_url(url)
    if make_tag == 'full':
        url = f'<A href="{url}">{url}</A>'
    if make_tag == 'start':
        url = f'<A href="{url}">'
    return url

In [39]:
def replace_url(txt, bb_uid):
    
    # simple URL
    txt = re.split(
       f'\[url\:{bb_uid}\]([^\[]+)\[\/url\:{bb_uid}\]',
       txt
    )
    txt = [ item if index%2 == 0 else url_tag(item, make_tag='full') for index, item in enumerate(txt)]
    txt = ''.join(txt)

    # complex URL
    open_tag = f'\[url\=([^\:]+)\:{bb_uid}\]'
    close_tag = f'\[\/url\:{bb_uid}\]'
    txt = re.split(
        open_tag,
        txt
    )
    txt = [ item if index%2 == 0 else url_tag(item, make_tag='start') for index, item in enumerate(txt)]
    txt = ''.join(txt)
    txt = re.sub(
        close_tag,
        '</A>',
        txt
    )

    return txt

In [113]:
def replace_size(txt, bb_uid):
    pattern_start = f'\[size=([0-9]+)\:{post_uid}]'
    pattern_end = f'\[\/size\:{bb_uid}]'
    txt = re.sub(
        pattern_start,
        r'<SPAN style="font-size:\1%">',
        txt
    )
    txt = re.sub(
        pattern_end,
        r'</SPAN>',
        txt
    )
    return txt
    

In [116]:
def repalce_color(txt, bb_uid):
    pattern_start = f'\[color=([\#a-zA-Z0-9]+)\:{bb_uid}\]'
    pattern_end = f'\[\/color\:{bb_uid}]'
    txt = re.sub(
        pattern_start,
        r'<SPAN style="color:\1">',
        txt
    )
    txt = re.sub(
        pattern_end,
        r'</SPAN>',
        txt
    )
    return txt

In [145]:
def img_tag(url):

    url = html.unescape(url)
    url_url = urllib.parse.quote(url)
    tag =f'''<P class="image_holder">
    <IMG src="{url}" />
    <A href="https://archive.org/search?query={url_url}">{url}</A>
    </P>'''
    return tag


In [146]:
def replace_img(txt, bb_uid):
   
    txt = re.split(
       f'\[img\:{bb_uid}\]([^\[]+)\[\/img\:{bb_uid}\]',
       txt
    )
    txt = [ item if index%2 == 0 else img_tag(item) for index, item in enumerate(txt)]
    txt = ''.join(txt)

    return txt

In [10]:
db = os.environ.get('PSQL_DATABASE')
host = 'localhost'
user ='user'
port = '5432'
psql_engine = f'postgresql://{user}:@{host}:{port}/{db}'

In [11]:
with open("get_single_post.sql", "r") as f:
    sql_template = f.readlines()

In [121]:
post_id = 37726
slq_statement = ''.join(sql_template).format(post_id = post_id)
df = pd.read_sql(slq_statement, psql_engine)
post_content = df.loc[0].to_dict()

In [122]:
post_meta = {k : v for k, v in post_content.items() if k not in ['post_text', 'bbcode_uid']}
post_text = post_content['post_text']
post_uid = post_content['bbcode_uid']

In [123]:
print(post_text)

[quote=&quot;Surion&quot;:1oxx7fpc][quote=&quot;7z9&quot;:1oxx7fpc][quote=&quot;Solaris&quot;:1oxx7fpc]To jest autentycznie genialny pomysł![/quote:1oxx7fpc]
masz na myśli zagładę Suriona?[/quote:1oxx7fpc]
Myślę jednak, że internet tak szybko nie zniknie. :&gt;[/quote:1oxx7fpc]
[i:1oxx7fpc]A jak zniknie? *[/i:1oxx7fpc]
[img:1oxx7fpc]http&#58;//imgs&#46;xkcd&#46;com/comics/the_cloud&#46;png[/img:1oxx7fpc]
*[size=85:1oxx7fpc]There's planned downtime every night when we turn on the Roomba and it runs over the cord.[/size:1oxx7fpc]


In [148]:
final_text = replace_linebreaks(post_text, post_uid)
final_text = replace_simple_tags(final_text, post_uid)
final_text = replace_quotes(final_text, post_uid)
final_text = replace_lists(final_text, post_uid)
final_text = replace_url(final_text, post_uid)
final_text = replace_size(final_text, post_uid)
final_text = repalce_color(final_text, post_uid)
final_text = replace_img(final_text, post_uid)
print(final_text)

<BLOCKQUOTE><P>
»Surion« pisze:<BR />
<BLOCKQUOTE><P>
»7z9« pisze:<BR />
<BLOCKQUOTE><P>
»Solaris« pisze:<BR />
To jest autentycznie genialny pomysł!
</P></BLOCKQUOTE>
masz na myśli zagładę Suriona?
</P></BLOCKQUOTE>
Myślę jednak, że internet tak szybko nie zniknie. :&gt;
</P></BLOCKQUOTE>
<EM>A jak zniknie? *</EM><BR />
<P class="image_holder">
    <IMG src="http://imgs.xkcd.com/comics/the_cloud.png" />
    <A href="https://archive.org/search?query=http%3A//imgs.xkcd.com/comics/the_cloud.png">http://imgs.xkcd.com/comics/the_cloud.png</A>
    </P><BR />
*<SPAN style="font-size:85%">There's planned downtime every night when we turn on the Roomba and it runs over the cord.</SPAN>
